# ITPS Flight Test Analysis Tool - v1.09
<p>Use this tool on FIREFOX and CHROME only </p>
<p>Chrome is going through updates and may have trouble downloading files</p>
<p>in this case, use Firefox</p>
<p>Safari and Edge have a few compatibility issues</p>

In [ ]:
# Data File Selection
from FTAT_imports import *

# make display 100% of the screen
display(HTML("<style>.container { width:100% !important; }</style>"))

# set this variable to True to bypass file import and generate fake data
unit_test = True

if not unit_test:
    f = FileBrowser()
    display(f.widget())  #
    #   <interact with widget, select a path>
    # in a separate cell:

In [ ]:
# Data Import


if unit_test:
    # creating fake data just to test
    rng = pd.date_range('27/10/2018 13:00:00', periods=5000, freq='50ms')
    raw_data = pd.DataFrame(data=rng, columns=['Time'])
    raw_data.set_index(['Time'], inplace=True)
    raw_data['counter'] = np.arange(len(raw_data))
    raw_data['fake_angle'] = raw_data.apply(lambda x: (raw_data['counter']/10)%(2*np.pi))
    raw_data['sin0'] = raw_data['counter'] #I have no idea why, but if I do not first create 'sine1' and then calculate with lambda, it throwns me an error
    raw_data['sin0'] = raw_data.apply(lambda x: np.sin(raw_data['fake_angle']))
    raw_data['cos0'] = raw_data['counter']
    raw_data['cos0'] = raw_data.apply(lambda x: np.cos(raw_data['fake_angle']))
    for i in range(50):
        sine_label = 'sin' + str(i+1)
        cosine_label = 'cos' + str(i+1)
        raw_data[sine_label] = raw_data['sin0']
        raw_data[cosine_label] = raw_data['cos0'] 
else:
    # select file and read from disk
    print('Preparing Graph Parameters ... may take a while ... patience is a virtue.')
    print('Reading data from {} ...'.format(f.path))
    #filepath='./IadsDataExport_clean.csv' #
    #raw_data=pd.read_csv(filepath, encoding='latin1', low_memory=False) #
    raw_data=pd.read_csv(f.path, encoding='latin1', low_memory=False) ##
    print('Done.')
    dirty_file = False
    if raw_data['Time'][10].count(':') > 2:
        dirty_file = True

    if dirty_file == True:
        ## FOR DIRTY DATA
        print('Dirty file detected ... cleaning up the data...')  #DIRTY DATA
        raw_data.fillna(value=0, inplace=True)  #DIRTY DATA
        raw_data['Time'] = (raw_data['Time'].str.slice_replace(0,4,''))  #DIRTY DATA

    raw_data['Time'] = pd.to_datetime(raw_data['Time'])   #CLEAN DATA
    raw_data = raw_data.set_index(['Time'])   #CLEAN DATA

print('Done.')
print('Imported data from: {} to: {}, with {} records'.format(raw_data.index[0], raw_data.index[-1], len(raw_data.index) ))

In [ ]:
# Main Program

# Let's subsample to speed up the graphing process
desired_sub_sample_rate = 5 #Hz
counter = 0
start_sample = raw_data.index[0]
next_sample = start_sample
while (next_sample - start_sample <= np.timedelta64(1, 's')):
    counter = counter + 1
    next_sample = raw_data.index[counter]
if desired_sub_sample_rate <= counter:
    sub_sample_rate = int(counter / desired_sub_sample_rate)
else:
    sub_sample_rate = int(counter)
    desired_sub_sample_rate = counter
    
sub_sampled_data = raw_data[::sub_sample_rate]

print()
print('Original data at {} Hz'.format((counter-1)))
print('Data subsampled to {} Hz to speed up graphing. Reduced to {} records'.format(desired_sub_sample_rate, len(sub_sampled_data.index)))
print('Done.')



time_slices_db = {} #this is the time slices data base, TP#, Description, Start, Stop (indexes)
selected_slice = [] #this is a global variable to store the currently selected slice that will be stored in the time_slices_db


######################### Slice Map and Definitions ##################################################
slicemap = ParameterMap(sub_sampled_data, 'MediumSeaGreen', 'Parameter Map')



######################### Slice Graph Definitions ####################################################

# data
current_plot_data = sub_sampled_data


slice_plot = LinePlotBrush(current_plot_data.index.values, current_plot_data.index.values)
detail_plot = LinePlot(current_plot_data.index.values, current_plot_data.index.values)
#slice_plot = LinePlotBrush(current_plot_data.index, current_plot_data.index)
#detail_plot = LinePlot(current_plot_data.index, current_plot_data.index)
detail_plot_stats = widgets.HTML(
    value="Empty <b>Empty</b>",
    placeholder='Stats',
    description='Stats',
)
detail_plot_stats.value = 'Empty'

# instantiate the slice selector
slicebox = sliceSelectDialog(current_plot_data)

######################### Parameter Selection and Detail Plot Updater #########################################


def detail_plot_update_brush(*args):
    lib_detail_plot_update_brush(*args, cpt=current_plot_data, selsl=selected_slice, detplt=detail_plot, slcplt=slice_plot,  slcbx=slicebox, dtstats=detail_plot_stats)

            
# link brush to slice plot
slice_plot.brushintsel.observe(detail_plot_update_brush, 'selected')



def selected_index_changed(change):
    '''
    This callback treats the parameter selection/deselection on the map
    Depending on the active tab, it will also callback the appropriate graph updater
    '''
    global current_plot_data, sub_sampled_data
    selected_tab = FTAT_tabs.selected_index
    if (selected_tab == 0 or selected_tab ==2):
        #this is for time slice or analysis tabs
        #define the current plot data per selected parameters
        if len(slicemap.map.selected) != 0:
            selected_parameters_plot_data = np.transpose(sub_sampled_data[slicemap.map.selected].values)
            current_plot_data = sub_sampled_data[slicemap.map.selected]
    if selected_tab == 0:
        #this is the time slice tab
        if len(slicemap.map.selected) != 0:
            slice_plot.line.y = selected_parameters_plot_data
            slice_plot.fig.title = str(slicemap.map.selected)
            detail_plot.line.y = selected_parameters_plot_data
            detail_plot.fig.title = str(slicemap.map.selected) + ' - Zoomed'
    elif selected_tab == 1:
        #this is the save to disk tab
        pass
    elif selected_tab == 2:
        #this is the analysis tab
        if len(slicemap.map.selected) != 0:
            analysis_plot.line.y = selected_parameters_plot_data
            analysis_plot.y_data = selected_parameters_plot_data  ##
            analysis_plot.fig.title = str(slicemap.map.selected)
            analysis_plot.ys.min = selected_parameters_plot_data.min()
            analysis_plot.ys.max = selected_parameters_plot_data.max()
            
            update_analysis_plot(current_plot_data, slicemap.map.selected,
                              analysis_plot,
                              poly_order, 
                              tz_slider.value,
                              analysis_plot_zoom_slider)
            
    elif selected_tab == 3:
        pass
    else:
        pass

# link selection handler to parameter map
slicemap.map.observe(selected_index_changed, 'selected')

############################### Plot Button Logic #########################
plot_button = widgets.Button(description='Plot Slice Below')

def on_plot_button_clicked(b):
    lib_on_plot_button_clicked(b, cpt=current_plot_data, detplt=detail_plot, slcbx=slicebox, dtstats=detail_plot_stats)
    
plot_button.on_click(on_plot_button_clicked)

########################## Save Slice Button and Logic ##############################

# time slices are saved to a dictionary for later analysis
TP_number_box = widgets.FloatText(
    value=0.0,
    description='TP#:',
    disabled=False,
    layout=Layout(width='130px')
)

TP_desc_box = widgets.Text(
    value='',
    placeholder='Type something',
    description='TP Title:',
    disabled=False
)
TS_message = Label()


################## Data Slice Selector Instance ####################
data_slice_selec = DataSliceSelect(time_slices_db)



################## Add/Remove Time Slices to db Logic #######################

def update_TP_saved_display(time_slices_db):
    display_text = 'TP#  Description \n'
    for key in time_slices_db:
        display_text = display_text  + str(key) + '  ' + time_slices_db[key][0] + '\n'
    return display_text


        
def on_saveTS_button_clicked(b):
    lib_on_saveTS_button_clicked(b, cpt=current_plot_data, detplt=detail_plot, slcplt=slice_plot, \
                                tsdb=time_slices_db, tpnb=TP_number_box, tsmsg=TS_message, \
                                tpdescb=TP_desc_box, tpsavdd=TP_saved_dd, tsdbrb=time_slices_db_radio)
        

def on_delTS_button_clicked(b):
    lib_on_delTS_button_clicked(b, tsdb=time_slices_db, tsmsg=TS_message, \
                                tpsavdd=TP_saved_dd, tsdbrb=time_slices_db_radio)

TP_saved_dd = widgets.Dropdown(
    options=time_slices_db,
    description='TP#:',
    disabled=False,
)


def on_TP_saved_dd_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        TS_message.value = ''

TP_saved_dd.observe(on_TP_saved_dd_change)
    
    
saveTS_button = widgets.Button(description='Save TSlice')
saveTS_button.on_click(on_saveTS_button_clicked)

delTS_button = widgets.Button(description='Delete TSlice')
delTS_button.on_click(on_delTS_button_clicked)

################## TS Radio Buttons ###################################################

time_slices_db_radio = widgets.RadioButtons(
    options=[],
    description='Slices:',
    disabled=False
)

def on_click_time_slices_db_radio(change):
    #everytime we select a slice, the analysis plot needs to be updated
    #using the parameters selected on the map and the TS from the dictionary
    global poly_order, current_plot_data, time_slices_db
    if (change['type'] == 'change') and (change['name'] == 'value') and (len(slicemap.map.selected) != 0) and (change['new'] != None):
        analysis_plot.x_data_slice_min = time_slices_db[change['new']][1]
        analysis_plot.x_data_slice_max = time_slices_db[change['new']][2]
        analysis_plot.update_plot(current_plot_data, slicemap.map.selected, time_slices_db[change['new']][1],
                                  time_slices_db[change['new']][2], poly_order, tz_slider.value)
        #here I need to update the zoomslider??
        #maybe not. if I just scale the graph...

time_slices_db_radio.observe(on_click_time_slices_db_radio, 'value')


################## Save Slices to Disk Logic ################################################
def save_slices(tp_dict):
    
    for key in tp_dict.keys():
        current_value = tp_dict[key]
        #use this for local tests
        #filepath = '~/Documents/'
        #use this one for jupyterlab
        filepath = '~/work/'
        filename1 = filepath + 'TP_' +  str(key) + '_' + current_value[0] + '.csv'
        filename2 = filepath + 'TP_' +  str(key) + '_' + current_value[0] + '_' + 'stats' + '.csv'

        #think where we can save this so the user can retrieve later...
        #remember we are in a container.
        slice_start = current_value[1]
        slice_end = current_value[2]
        sliced_data = raw_data.iloc[(raw_data.index >= slice_start) & 
                                        (raw_data.index <= slice_end)]
        sliced_data_stats = pd.DataFrame([sliced_data.mean(), sliced_data.std()], index=['mean','std'])

        if save_all_parameters.value:
            sliced_data.to_csv(filename1)
            sliced_data_stats.to_csv(filename2)
        else:
            sliced_data[slicemap.map.selected].to_csv(filename1)
            sliced_data_stats[slicemap.map.selected].to_csv(filename2)

    # zipping it all
    zip_filename = filepath+'TP_zipped.zip'
    zip_file_wildcard = filepath+'TP*.csv'
    !rm $zip_filename
    !zip $zip_filename $zip_file_wildcard
    !rm $zip_file_wildcard
    save_feedback.value = 'files saved to disk'
        
def on_save_slices_button_clicked(b):
    save_slices(time_slices_db)



save_instructions = widgets.Label(
    value='Select parameters to save or select checkbox below to save all',
    disable=False
    )

save_feedback = widgets.Label(
    value='',
    disable=False
    )

save_all_parameters = widgets.Checkbox(
    value=False,
    description='save ALL PARAMETERS',
    disabled=False
)

save_slices_button = widgets.Button(description='Save Slices to Disk')
save_slices_button.on_click(on_save_slices_button_clicked)



################# Analysis Plot Definition #####################

#analysis_plot = AnalysisPlot(current_plot_data.index.values, current_plot_data.index.values, time_slices_db)


#timezone slider to cope with ipywidgets/bqplot issue
tz_slider = widgets.IntSlider(
    value=-4,
    min=-6,
    max=6,
    step=1,
    description='TimeZone:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

def on_change_tz_slider(chage):
    update_analysis_plot(current_plot_data, slicemap.map.selected,
                              analysis_plot,
                              poly_order, 
                              tz_slider.value,
                              analysis_plot_zoom_slider)
    
tz_slider.observe(on_change_tz_slider)

tz_msg2 = widgets.Label('Adjust TZ Slider on other side if graph is blank')

analysis_plot = AnalysisPlot(current_plot_data.index, current_plot_data.index.values, tz_slider.value)
poly_order = 1

analysis_poly_order_dd = widgets.Dropdown(
            options=[1,2,3,4],
            description='Poly Deg',
            value=1,
            disabled=False
        )



def on_change_analysis_poly_order_dd(change):
    global poly_order, current_plot_data
    if change['type'] == 'change' and change['name'] == 'value' and len(slicemap.map.selected) != 0:
        poly_order = change['new']
        
        
        update_analysis_plot(current_plot_data, slicemap.map.selected,
                              analysis_plot,
                              poly_order, 
                              tz_slider.value,
                              analysis_plot_zoom_slider)
    

            
analysis_poly_order_dd.observe(on_change_analysis_poly_order_dd)

analysis_plot_zoom_slider = SimpleZoomSlider(analysis_plot)

def on_analysis_zoom_slider_change(change):
    
    update_analysis_plot(current_plot_data, slicemap.map.selected,
                              analysis_plot,
                              poly_order, 
                              tz_slider.value,
                              analysis_plot_zoom_slider)

analysis_plot_zoom_slider.zoom_slider.observe(on_analysis_zoom_slider_change, names='value')

# slice trim to zoomed figure logic
def slice_trim(tp_dict, key_to_find, slice_start, slice_end):
    for key in tp_dict.keys():
        if key == key_to_find:
            current_value = tp_dict[key]
            current_value[1] = slice_start
            current_value[2] = slice_end
            tp_dict[key] = current_value


def on_slice_trim_button_clicked(b):
    slice_trim(time_slices_db, time_slices_db_radio.value, np.datetime64(analysis_plot.xs.min, 'us'), np.datetime64(analysis_plot.xs.max, 'us'))
    
slice_trim_button = widgets.Button(description='Trim Current Slice')
slice_trim_button.on_click(on_slice_trim_button_clicked)



################# Strip Charts Definition #####################
strip_chart_button = widgets.Button(description='Plot Strip Chart Below')


strip_chart = StripChart(raw_data, slicemap, time_slices_db, time_slices_db_radio)


#####################################################################################################
# MAIN TABS DEFINITIONS
#####################################################################################################

tab_contents = ['Slice', 'Save', 'Analysis', 'StripChart']
text_children = [widgets.Text(description=name) for name in tab_contents]

slice_tab_children = VBox([slice_plot.fig, 
                           slicebox.slice_box,
                           plot_button,
                           detail_plot.fig, 
                           VBox([TS_message, HBox([saveTS_button, TP_number_box, TP_desc_box, TP_saved_dd, delTS_button])])
                          ]) 

save_tab_children = VBox([save_instructions, 
                          save_all_parameters, 
                          save_slices_button, 
                          save_feedback])

plot_tab_children = VBox([analysis_poly_order_dd, 
                          HBox([analysis_plot.fig, analysis_plot.fit_statistics]), 
                          analysis_plot_zoom_slider.zoom_slider, 
                          slice_trim_button,
                          tz_msg2])

strip_chart_children = strip_chart.widget()

complete_tabs_children = [slice_tab_children, save_tab_children, plot_tab_children, strip_chart_children]
FTAT_tabs = widgets.Tab()
FTAT_tabs.children = complete_tabs_children
for i in range(len(complete_tabs_children)):
    FTAT_tabs.set_title(i, tab_contents[i])
FTAT_tabs.layout = Layout(width='100%')

# linking parameter and tab selections
# callbacks

def update_logic(param01):
    selected_tab = FTAT_tabs.selected_index
    #reset the saved files feedback everytime the tabs as changed
    save_feedback.value = ''
    if selected_tab == 0:
        selected_index_changed(None)
    elif selected_tab == 1:
        pass
    elif selected_tab == 2:
        selected_index_changed(None)
    elif selected_tab == 3:
        pass
    else :
        pass
       

def on_FTAT_tabs_change(change):
    #this is just a wrapper function that will allow me to pass other parameters/objects
    #for now, just call the updater
    update_logic(1)
    
FTAT_tabs.observe(on_FTAT_tabs_change)


HBox([VBox(children = [slicemap.map, tz_slider, time_slices_db_radio], layout=Layout(width='40%')), 
      Box(children = [FTAT_tabs])])

In [ ]:
detail_plot.xs.min

In [ ]:
selected_slice[0]